In [ ]:
import pandas as pd
import numpy as np
import talib as ta
import mplfinance as mpf
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("zijin.csv")
df = df[df["tradestatus"] == 1]
df = df[["Date","Open","High","Low","Close"]]

In [ ]:
df["ln"] = df["Low"].rolling(9).min()
df["hn"] = df["High"].rolling(9).max()
df["rsv"] = (df["Close"] - df["ln"])/(df["hn"] - df["ln"]) * 100

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df.index = df["Date"]
df["K"] = 50 
df["D"] = 50 
df["J"] = 50 

for i in range(10,len(df["Date"])):
    df["K"].iloc[i] = 2/3*df["K"].iloc[i-1]+df["rsv"].iloc[i]/3
    
for i in range(10,len(df["Date"])):
    df["D"].iloc[i] = 2/3*df["D"].iloc[i-1]+df["K"].iloc[i]/3
    
df["J"] = 3*df["K"] - 2*df["D"]

In [ ]:
fig = plt.figure()
fig.set_size_inches((25, 18))
kdj = fig.add_axes((0, 0.24, 1, 0.5))
kdj.plot(df.index, df["K"], label="K")
kdj.plot(df.index, df["D"], label="D")
kdj.plot(df.index, df["J"], label="J")
kdj.plot(df.index, [90] * len(df.index), label="K_overbought")
kdj.plot(df.index, [80] * len(df.index), label="D_overbought")
kdj.plot(df.index, [50] * len(df.index), label="no trend")
kdj.plot(df.index, [20] * len(df.index), label="D_oversold")
kdj.plot(df.index, [10] * len(df.index), label="K_oversold")

kdj.legend()

In [ ]:
df["position"] = None 
df["trade"] = None 
bd = []
bp = []
sd = []
sp = []
for i in range(len(df.index)): 
    if df["D"].iloc[i-1] > df["K"].iloc[i-1] and df["D"].iloc[i] < df["K"].iloc[i] and df["D"].iloc[i]<30 :
        df["position"].iloc[i] = 'oversold'
    elif df["D"].iloc[i-1] < df["K"].iloc[i-1] and df["D"].iloc[i] > df["K"].iloc[i] and df["D"].iloc[i]>70 :
        df["position"].iloc[i] = 'overbought'
    else:
        df["position"].iloc[i] = None
for i in range(len(df["position"])):
    if df["position"].iloc[i-1] == 'oversold' and df["position"].iloc[i] == None:
        df["trade"].iloc[i] = "buy"
        bd.append(df["Date"].iloc[i])
        bp.append(df["Close"].iloc[i])
    if df["position"].iloc[i-1] == 'overbought' and df["position"].iloc[i] == None:
        df["trade"].iloc[i] = "sell"
        sd.append(df["Date"].iloc[i])
        sp.append(df["Close"].iloc[i])

In [ ]:
fig = plt.figure()
fig.set_size_inches((25, 18))
kline = fig.add_axes((0, 0.24, 1, 0.5))
kline.plot(df.index, df["Close"], label = "K line")
kline.plot(bd,bp,"ro", label = "buy position")
kline.plot(sd,sp,"ks", label = "sell position")

kline.legend()